In [25]:
import pandas as pd
bat2019 = pd.read_csv('Batting2019.csv', sep = ',', error_bad_lines=False)
pitch2019 = pd.read_csv('Pitching2019.csv', sep = ',', error_bad_lines=False)
field2019 = pd.read_csv('Fielding2019.csv', sep = ',', error_bad_lines=False)

In [28]:
print(bat2019.columns)
print(pitch2019.columns)
print(field2019.columns)
batandField = pd.merge(bat2019, field2019, on ='Name', how = 'inner' )
print(batandField.columns)

Index(['Name', 'Team', 'G', 'PA', 'HR', 'R', 'RBI', 'SB', 'BB%', 'K%', 'ISO',
       'BABIP', 'AVG', 'OBP', 'SLG', 'wOBA', 'wRC+', 'BsR', 'Off', 'Def',
       'WAR', 'playerid'],
      dtype='object')
Index(['Name', 'Team', 'W', 'L', 'SV', 'G', 'GS', 'IP', 'K/9', 'BB/9', 'HR/9',
       'BABIP', 'LOB%', 'GB%', 'HR/FB', 'ERA', 'FIP', 'xFIP', 'WAR',
       'playerid'],
      dtype='object')
Index(['Name', 'Team', 'Pos', 'Inn', 'rSZ', 'rCERA', 'rSB', 'rGDP', 'rARM',
       'rGFP', 'rPM', 'rTS', 'DRS', 'BIZ', 'Plays', 'RZR', 'OOZ', 'FSR', 'FRM',
       'ARM', 'DPR', 'RngR', 'ErrR', 'UZR', 'UZR/150', 'Def', 'playerid'],
      dtype='object')
Index(['Name', 'Team_x', 'G', 'PA', 'HR', 'R', 'RBI', 'SB', 'BB%', 'K%', 'ISO',
       'BABIP', 'AVG', 'OBP', 'SLG', 'wOBA', 'wRC+', 'BsR', 'Off', 'Def_x',
       'WAR', 'playerid_x', 'Team_y', 'Pos', 'Inn', 'rSZ', 'rCERA', 'rSB',
       'rGDP', 'rARM', 'rGFP', 'rPM', 'rTS', 'DRS', 'BIZ', 'Plays', 'RZR',
       'OOZ', 'FSR', 'FRM', 'ARM', 'DPR', 'RngR', 

In [29]:
print(batandField[['Def_x', 'Def_y']])

     Def_x  Def_y
0     -0.4    0.1
1     -0.1    0.0
2     -1.5   -1.5
3      0.7    0.7
4      0.1    0.1
5      2.3    2.3
6      1.3    1.3
7     -1.5   -1.5
8      0.4    0.4
9      2.0    2.0
10    -0.3   -0.3
11    -0.1   -0.1
12     0.0    0.2
13     0.9    0.9
14     0.8    0.9
15     1.0    1.0
16     2.5    2.5
17     0.6    0.6
18    -0.4   -0.3
19    -3.3   -3.3
20     0.0    0.0
21    -0.4   -0.4
22     1.9    1.9
23     1.5    1.5
24    -0.5   -0.5
25     1.8    1.8
26    -0.6   -0.6
27     2.7    2.7
28     2.5    2.5
29     0.3    0.3
..     ...    ...
128   -0.5   -0.5
129   -4.9   -4.9
130    1.1    1.1
131    0.6    0.6
132    0.0    0.0
133   -1.8   -1.8
134   -1.1   -0.9
135    0.9    0.9
136    0.7    0.7
137    1.1    1.1
138    0.2   -1.1
139   -2.2   -2.1
140   -1.8   -1.8
141   -1.9   -1.5
142   -1.8   -1.8
143   -1.1   -1.1
144   -1.3   -1.3
145   -0.1   -0.1
146   -1.4   -1.4
147   -1.1   -1.1
148    0.1    0.1
149    0.0    0.0
150    1.0    1.0
151   -2.1